In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import pandas as pd
import numpy as np

data4 = pd.read_csv("/content/drive/MyDrive/КНИР Васильев Игорь БПМ19-4/Разные модели/sqsq.csv")
data4 = data4.drop(data4[data4.Label == 'Label'].index)

NORM_LEN = 2000
DDOS_LEN = 2000
BRUTE_LEN = 1500
SQL_LEN = 2500
data1 = pd.read_csv("/content/drive/MyDrive/КНИР Васильев Игорь БПМ19-4/Разные модели/norm.csv")
data2 = pd.read_csv("/content/drive/MyDrive/КНИР Васильев Игорь БПМ19-4/Разные модели/ddos.csv")
data3 = pd.read_csv("/content/drive/MyDrive/КНИР Васильев Игорь БПМ19-4/Разные модели/nb1.csv")
data1 = data1.sample(NORM_LEN)
data2 = data2.sample(DDOS_LEN)
data3 = data3.sample(BRUTE_LEN)
df = data4.sample(SQL_LEN)
df = df.loc[df['Label'] == 'Infilteration']
df2 = data4.sample(1500)
data = pd.concat([data1, data2, data3, df, df2], ignore_index=True, sort=False)
shuffle = data.sample(frac=1, random_state=101).reset_index()

<ipython-input-5-9c976e5d1777>:4: DtypeWarning: Columns (0,1,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78) have mixed types. Specify dtype option on import or set low_memory=False.
  data4 = pd.read_csv("/content/drive/MyDrive/КНИР Васильев Игорь БПМ19-4/Разные модели/sqsq.csv")


In [6]:
shuffle.fillna(0, inplace=True)
df = shuffle

In [7]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [8]:
test_df = df.copy()

In [9]:
test_df.replace([np.inf, -np.inf], np.nan, inplace=True)
test_df.dropna(how="all", inplace=True)

number_inf = test_df[test_df == np.inf].count()

In [10]:
def ip2int(ip):
    if ip == 0:
      ip = '0.0.0.0'
    ip_list = list(map(int, ip.split('.')))
    num = 0
    for i in ip_list:
        num *= 256
        num += i
    return num

In [11]:
def delete_first_zero(val):
    if val[0] == '0':
        return val[1:]
    return val

def time2int(tm):
    tm_split = tm.split(' ')
    day_info = tm_split[0].split('/')
    day_info = list(map(delete_first_zero, day_info))
    day_info = list(map(int, day_info))
    day_amount = day_info[0] + (day_info[1] - 1) * 30 + (day_info[2] - 1970) * 365
    
    time_info = tm_split[1].split(':')
    time_info = list(map(delete_first_zero, time_info))
    time_info = list(map(int, time_info))
    sec_amount = 60 * (time_info[0] * 60 + time_info[1]) + time_info[2]
    
    if len(tm_split) == 3 and tm_split[-1] == 'PM':
        sec_amount += 3600 * 12
    sec_amount += day_amount * 24 * 3600
    
    return sec_amount

In [12]:
y = test_df['Label']
test_df = test_df.drop(['Label'], axis=1)
test_df['Src IP'] = test_df['Src IP'].apply(ip2int)
test_df['Dst IP'] = test_df['Dst IP'].apply(ip2int)
test_df['Timestamp'] = test_df['Timestamp'].apply(time2int)
test_df.head()

,index,Unnamed: 0,Flow ID,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,Bwd Pkt Len Min,Bwd Pkt Len Mean,Bwd Pkt Len Std,Flow Byts/s,Flow Pkts/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Tot,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Tot,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Len,Bwd Header Len,Fwd Pkts/s,Bwd Pkts/s,Pkt Len Min,Pkt Len Max,Pkt Len Mean,Pkt Len Std,Pkt Len Var,FIN Flag Cnt,SYN Flag Cnt,RST Flag Cnt,PSH Flag Cnt,ACK Flag Cnt,URG Flag Cnt,CWE Flag Count,ECE Flag Cnt,Down/Up Ratio,Pkt Size Avg,Fwd Seg Size Avg,Bwd Seg Size Avg,Fwd Byts/b Avg,Fwd Pkts/b Avg,Fwd Blk Rate Avg,Bwd Byts/b Avg,Bwd Pkts/b Avg,Bwd Blk Rate Avg,Subflow Fwd Pkts,Subflow Fwd Byts,Subflow Bwd Pkts,Subflow Bwd Byts,Init Fwd Win Byts,Init Bwd Win Byts,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,2300,176044.0,192.168.2.110-69.84.133.138-4339-80-6,3232236142,4339.0,1163167114,80,6,1275536144,37,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,54054.054054,37.0,0.0,37.0,37.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,20,20,27027.027027,27027.027027,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,1.0,0.0,0.0,0.0,0,0,0,0,0,0,1,0,1,0,-1,64860,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4503,0.0,0,0,0.0,0,21,6,1517568866,1,1,1,0,0,0,0,0.0,0.0,0,0,0.0,0.0,0.0,2000000.0,1.0,0.0,1,1,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,40,20,1000000.0,1000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,26883.0,0.0,0.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,7181,0.0,0,0,0.0,0,53,17,1519002843,279,1,1,32,128,32,32,32.0,0.0,128,128,128.0,0.0,573476.7025,7168.458781,279.0,0.0,279,279,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,8,8,3584.229391,3584.229391,32,128,64.0,55.425626,3072.0,0,0,0,0,0,0,0,0,1,96.0,32.0,128.0,0,0,0,0,0,0,1,32,1,128,-1,-1,0,8,0.0,0.0,0,0,0.0,0.0,0,0
3,4633,0.0,0,0,0.0,0,21,6,1517570389,2,1,1,0,0,0,0,0.0,0.0,0,0,0.0,0.0,0.0,1000000.0,2.0,0.0,2,2,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,40,20,500000.0,500000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,26883.0,0.0,0.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1900,5249435.0,172.31.68.13-209.85.203.156-49730-443-6,2887730189,49730.0,3512060828,443,6,1518078583,550830,9,6,518.0,2877.0,200.0,0.0,57.555556,84.729144,1430.0,0.0,479.5,624.120421,6163.426102,27.231632,39345.0,68589.61797,233863.0,5.0,550830.0,68853.75,80157.734652,233863.0,15.0,221328.0,44265.6,61348.96609,125268.0,20.0,0,0,0,0,192,132,16.338979,10.892653,0.0,1430.0,212.1875,423.794167,179601.495833,0,0,1,1,0,0,0,1,0.0,226.333333,57.555556,479.5,0,0,0,0,0,0,9,518,6,2877,8192,172,4,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


IMPORTANT_COLS = ['Src IP',
                  'Dst IP',
                  'Timestamp',
                  'Src Port',
                  'Dst Port',
                  'Protocol','Flow Duration',
                  'Flow Byts/s',
                  'Flow Pkts/s',
                  'Pkt Len Var',
                  'Down/Up Ratio'
                 ]

In [13]:
test_df = test_df.drop(['Flow ID'], axis=1)
test_df.head()

,index,Unnamed: 0,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,Bwd Pkt Len Min,Bwd Pkt Len Mean,Bwd Pkt Len Std,Flow Byts/s,Flow Pkts/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Tot,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Tot,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Len,Bwd Header Len,Fwd Pkts/s,Bwd Pkts/s,Pkt Len Min,Pkt Len Max,Pkt Len Mean,Pkt Len Std,Pkt Len Var,FIN Flag Cnt,SYN Flag Cnt,RST Flag Cnt,PSH Flag Cnt,ACK Flag Cnt,URG Flag Cnt,CWE Flag Count,ECE Flag Cnt,Down/Up Ratio,Pkt Size Avg,Fwd Seg Size Avg,Bwd Seg Size Avg,Fwd Byts/b Avg,Fwd Pkts/b Avg,Fwd Blk Rate Avg,Bwd Byts/b Avg,Bwd Pkts/b Avg,Bwd Blk Rate Avg,Subflow Fwd Pkts,Subflow Fwd Byts,Subflow Bwd Pkts,Subflow Bwd Byts,Init Fwd Win Byts,Init Bwd Win Byts,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,2300,176044.0,3232236142,4339.0,1163167114,80,6,1275536144,37,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,54054.054054,37.0,0.0,37.0,37.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,20,20,27027.027027,27027.027027,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,1.0,0.0,0.0,0.0,0,0,0,0,0,0,1,0,1,0,-1,64860,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,4503,0.0,0,0.0,0,21,6,1517568866,1,1,1,0,0,0,0,0.0,0.0,0,0,0.0,0.0,0.0,2000000.0,1.0,0.0,1,1,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,40,20,1000000.0,1000000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,26883.0,0.0,0.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,7181,0.0,0,0.0,0,53,17,1519002843,279,1,1,32,128,32,32,32.0,0.0,128,128,128.0,0.0,573476.7025,7168.458781,279.0,0.0,279,279,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,8,8,3584.229391,3584.229391,32,128,64.0,55.425626,3072.0,0,0,0,0,0,0,0,0,1,96.0,32.0,128.0,0,0,0,0,0,0,1,32,1,128,-1,-1,0,8,0.0,0.0,0,0,0.0,0.0,0,0
3,4633,0.0,0,0.0,0,21,6,1517570389,2,1,1,0,0,0,0,0.0,0.0,0,0,0.0,0.0,0.0,1000000.0,2.0,0.0,2,2,0,0.0,0.0,0,0,0,0.0,0.0,0,0,0,0,0,0,40,20,500000.0,500000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,26883.0,0.0,0.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1900,5249435.0,2887730189,49730.0,3512060828,443,6,1518078583,550830,9,6,518.0,2877.0,200.0,0.0,57.555556,84.729144,1430.0,0.0,479.5,624.120421,6163.426102,27.231632,39345.0,68589.61797,233863.0,5.0,550830.0,68853.75,80157.734652,233863.0,15.0,221328.0,44265.6,61348.96609,125268.0,20.0,0,0,0,0,192,132,16.338979,10.892653,0.0,1430.0,212.1875,423.794167,179601.495833,0,0,1,1,0,0,0,1,0.0,226.333333,57.555556,479.5,0,0,0,0,0,0,9,518,6,2877,8192,172,4,20,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
test_df = test_df.fillna(0)

In [15]:
import numpy as np

test_df = test_df.astype(float)

In [16]:
df_z_scaled = test_df.copy()
  
# apply normalization techniques
for column in df_z_scaled.columns:
    df_z_scaled[column] = (df_z_scaled[column] -
                           df_z_scaled[column].mean()) / df_z_scaled[column].std()
df_z_scaled = df_z_scaled.fillna(0)
df_z_scaled.head()

,index,Unnamed: 0,Src IP,Src Port,Dst IP,Dst Port,Protocol,Timestamp,Flow Duration,Tot Fwd Pkts,Tot Bwd Pkts,TotLen Fwd Pkts,TotLen Bwd Pkts,Fwd Pkt Len Max,Fwd Pkt Len Min,Fwd Pkt Len Mean,Fwd Pkt Len Std,Bwd Pkt Len Max,Bwd Pkt Len Min,Bwd Pkt Len Mean,Bwd Pkt Len Std,Flow Byts/s,Flow Pkts/s,Flow IAT Mean,Flow IAT Std,Flow IAT Max,Flow IAT Min,Fwd IAT Tot,Fwd IAT Mean,Fwd IAT Std,Fwd IAT Max,Fwd IAT Min,Bwd IAT Tot,Bwd IAT Mean,Bwd IAT Std,Bwd IAT Max,Bwd IAT Min,Fwd PSH Flags,Bwd PSH Flags,Fwd URG Flags,Bwd URG Flags,Fwd Header Len,Bwd Header Len,Fwd Pkts/s,Bwd Pkts/s,Pkt Len Min,Pkt Len Max,Pkt Len Mean,Pkt Len Std,Pkt Len Var,FIN Flag Cnt,SYN Flag Cnt,RST Flag Cnt,PSH Flag Cnt,ACK Flag Cnt,URG Flag Cnt,CWE Flag Count,ECE Flag Cnt,Down/Up Ratio,Pkt Size Avg,Fwd Seg Size Avg,Bwd Seg Size Avg,Fwd Byts/b Avg,Fwd Pkts/b Avg,Fwd Blk Rate Avg,Bwd Byts/b Avg,Bwd Pkts/b Avg,Bwd Blk Rate Avg,Subflow Fwd Pkts,Subflow Fwd Byts,Subflow Bwd Pkts,Subflow Bwd Byts,Init Fwd Win Byts,Init Bwd Win Byts,Fwd Act Data Pkts,Fwd Seg Size Min,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
0,-0.699997,-0.420856,1.226699,-0.572965,-0.067111,-0.320924,-0.453624,-1.900424,-0.398445,-0.069224,-0.069864,-0.015838,-0.043908,-0.522096,-0.369176,-0.600092,-0.525605,-0.563935,-0.369054,-0.506111,-0.514962,-0.038415,0.0,-0.227928,-0.321247,-0.369823,-0.154611,-0.355558,-0.234799,-0.295452,-0.322941,-0.114992,-0.312635,-0.222631,-0.292004,-0.295098,-0.078945,-0.194675,-0.059231,-0.039449,0.0,-0.110801,-0.082155,-0.362004,-0.375936,-0.372988,-0.584902,-0.549878,-0.573960,-0.342529,-0.162631,-0.509379,-0.331823,-0.846564,1.888403,-0.154477,-0.039449,-0.330378,0.353425,-0.582512,-0.600092,-0.506111,0.0,0.0,0.0,0.0,0.0,0.0,-0.069224,-0.015838,-0.069864,-0.043908,-0.628421,2.804905,-0.036634,-1.140150,-0.116579,-0.080818,-0.128314,-0.099425,-0.317933,-0.169681,-0.329681,-0.298748
1,0.288208,-0.506590,-0.919763,-0.749433,-0.910417,-0.325254,-0.453624,0.522685,-0.398447,-0.069224,-0.069864,-0.015838,-0.043908,-0.522096,-0.369176,-0.600092,-0.525605,-0.563935,-0.369054,-0.506111,-0.514962,-0.038415,0.0,-0.227932,-0.321247,-0.369826,-0.154615,-0.355558,-0.234799,-0.295452,-0.322941,-0.114992,-0.312635,-0.222631,-0.292004,-0.295098,-0.078945,-0.194675,-0.059231,-0.039449,0.0,-0.086387,-0.082155,2.755455,3.142225,-0.372988,-0.584902,-0.549878,-0.573960,-0.342529,-0.162631,-0.509379,-0.331823,1.181093,-0.529479,-0.154477,-0.039449,-0.330378,0.353425,-0.582512,-0.600092,-0.506111,0.0,0.0,0.0,0.0,0.0,0.0,-0.069224,-0.015838,-0.069864,-0.043908,1.585474,-0.428997,-0.036634,1.639843,-0.116579,-0.080818,-0.128314,-0.099425,-0.317933,-0.169681,-0.329681,-0.298748
2,1.489485,-0.506590,-0.919763,-0.749433,-0.910417,-0.322905,2.089051,0.537042,-0.398437,-0.069224,-0.069864,-0.015307,-0.042433,-0.389124,1.124991,-0.074184,-0.525605,-0.341103,2.054806,0.070229,-0.514962,0.037449,0.0,-0.227902,-0.321247,-0.369807,-0.154584,-0.355558,-0.234799,-0.295452,-0.322941,-0.114992,-0.312635,-0.222631,-0.292004,-0.295098,-0.078945,-0.194675,-0.059231,-0.039449,0.0,-0.125449,-0.091646,-0.437117,-0.460703,1.121868,-0.364595,-0.088174,-0.295328,-0.322524,-0.162631,-0.509379,-0.331823,-0.846564,-0.529479,-0.154477,-0.039449,-0.330378,0.353425,0.064311,-0.074184,0.070229,0.0,0.0,0.0,0.0,0.0,0.0,-0.069224,-0.015307,-0.069864,-0.042433,-0.628421,-0.429046,-0.036634,-0.584151,-0.116579,-0.080818,-0.128314,-0.099425,-0.317933,-0.169681,-0.329681,-0.298748
3,0.346522,-0.506590,-0.919763,-0.749433,-0.910417,-0.325254,-0.453624,0.522701,-0.398447,-0.069224,-0.069864,-0.015838,-0.043908,-0.522096,-0.369176,-0.600092,-0.525605,-0.563935,-0.369054,-0.506111,-0.514962,-0.038415,0.0,-0.227932,-0.321247,-0.369826,-0.154615,-0.355558,-0.234799,-0.295452,-0.322941,-0.114992,-0.312635,-0.222631,-0.292004,-0.295098,-0.078945,-0.194675,-0.059231,-0.039449,0.0,-0.086387,-0.082155,1.153427,1.334281,-0.372988,-0.584902,-0.549878,-0.573960,-0.342529,-0.162631,-0.509379,-0.331823,1.18

In [17]:
y = y.str.replace('ddos', '1').str.replace('Benign','0').str.replace('FTP-BruteForce','2').str.replace('Infilteration','3')
y = y.astype(np.int64)

In [18]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_z_scaled, y, test_size=0.3, random_state=42)

In [19]:
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier

params = {
    'hidden_layer_sizes': [(100,1), (100,2), (100,3)],
    'activation': ['tanh', 'relu',"logistic"],
    'solver': ['sgd', 'adam'],
    'alpha': [10.0 ** -np.arange(1, 7)],
    'learning_rate': ["constant", "invscaling", "adaptive"]
}
params = {
    'hidden_layer_sizes': [(50,50,50)],
    'activation': ['tanh', 'relu',"logistic"],
    'solver': ['adam'],
    'alpha': [1e-5],
    'learning_rate': ["constant", "invscaling", "adaptive"]
}
clf = MLPClassifier(solver='adam', alpha=1e-5, random_state=1)
gridsearch = GridSearchCV(clf, params,return_train_score=True, refit=True)
gridsearch.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptro

GridSearchCV(estimator=MLPClassifier(alpha=1e-05, random_state=1),
             param_grid={'activation': ['tanh', 'relu', 'logistic'],
                         'alpha': [1e-05], 'hidden_layer_sizes': [(50, 50, 50)],
                         'learning_rate': ['constant', 'invscaling',
                                           'adaptive'],
                         'solver': ['adam']},
             return_train_score=True)

In [20]:
gridsearch.best_params_

{'activation': 'tanh',
 'alpha': 1e-05,
 'hidden_layer_sizes': (50, 50, 50),
 'learning_rate': 'constant',
 'solver': 'adam'}

In [21]:
y_true, y_pred = y_test , gridsearch.predict(X_test)
from sklearn.metrics import classification_report
print('Results on the test set:')
print(classification_report(y_true, y_pred))

Results on the test set:
              precision    recall  f1-score   support

           0       0.90      0.95      0.92       920
           1       1.00      1.00      1.00       591
           2       0.99      1.00      1.00       467
           3       0.84      0.71      0.77       339

    accuracy                           0.93      2317
   macro avg       0.93      0.91      0.92      2317
weighted avg       0.93      0.93      0.93      2317



In [45]:
y_true.value_counts()
np.array([1 for i in y_pred if i == '3']).sum()

0.0